# 04a - Vertex AI > Notebooks - Models Built in Notebooks with Tensorflow

Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

This notebook shows training a model directly within the runtime of the notebook environment.  Then the model is saved and moved to GCS for deployment to a Vertex AI Endpoint for online predictions.  The model training is done with [Tensorflow](https://www.tensorflow.org/), specifically [Keras](https://keras.io/), and was designed to show a neural network approach to logistic regression.  The training data batches are read from BigQuery using [Tensorflow I/O](https://www.tensorflow.org/io).

**Prerequisites:**

-  01 - BigQuery - Table Data Source

**Overview:**

-  Use Python Client for BigQuery
   -  Read the tables schema from BigQuery INFORMATION_SCHEMA
   -  Prepare the feature information for Tensorflow
-  Define a function that remaps the input data into features and target variables where target is one-hot encoded (classification model with 10 classes)
-  Set Tensorflow I/O read session
-  Demonstrate reading a single batch
-  Train a Tensorflow model
   -  Define the model layers
   -  Compile the model
   -  Fit the model
   -  Evaluate the model (loss, accuracy)
   -  Create prediction with the model
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Upload Model
      -  Model - aiplatform.Model.upoad
   -  Create Endpoint
      -  Endpoint - aiplatform.Endpoint.create
   -  Deploy to Endpoint
      -  Endpoint.deploy(model=Model)
   -  Online Predictions
      -  Endpoint.predict
-  Online Predictions with:
   -  REST call
   -  gcloud CLI

**Resources:**

-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)

**Related Training:**

-  todo

---
## Conceptual Architecture

<img src="architectures/statmike-mlops-04.png">

---
## Setup

inputs:

In [13]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'digits'
NOTEBOOK = '04a'

# Resources
DEPLOY_COMPUTE = 'n1-standard-4'
DEPLOY_IMAGE='us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-2:latest'

# Model Training
VAR_TARGET = 'target'
VAR_OMIT = 'target_OE' # add more variables to the string with space delimiters
EPOCHS = 25
BATCH_SIZE = 30

packages:

In [14]:
from google.cloud import bigquery

from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf

from google.cloud import aiplatform
from datetime import datetime

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [15]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [16]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

environment:

In [17]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get The Schema of The Training Data

In [18]:
query = f"SELECT * FROM {DATANAME}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{DATANAME}_prepped'"
schema = bigquery.query(query).to_dataframe()

In [19]:
schema

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position
0,statmike-mlops,digits,digits_prepped,p0,1,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
1,statmike-mlops,digits,digits_prepped,p1,2,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
2,statmike-mlops,digits,digits_prepped,p2,3,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
3,statmike-mlops,digits,digits_prepped,p3,4,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
4,statmike-mlops,digits,digits_prepped,p4,5,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,statmike-mlops,digits,digits_prepped,p62,63,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
63,statmike-mlops,digits,digits_prepped,p63,64,YES,FLOAT64,NEVER,None,None,NO,None,NO,NO,NaN
64,statmike-mlops,digits,digits_prepped,target,65,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN
65,statmike-mlops,digits,digits_prepped,target_OE,66,YES,STRING,NEVER,None,None,NO,None,NO,NO,NaN


## Prepare Inputs For Tensorflow Training

Use the the table schema to prepare the TensorFlow Model:
- Omit unused columns
- Create `feature_columns` for the model
- Define the `dtypes` for TensorFlow

In [21]:
OMIT = VAR_OMIT.split() + ['splits']

selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != VAR_TARGET:
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

# all the columns in this data source are either float64 or int64
output_types = schema[~schema.column_name.isin(OMIT)].data_type.tolist()
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

Define a function that remaps the input data for TensorFlow into:
- features
- `target` - and one_hot encoded for multi-class classification

In [22]:
def transTable(row_dict):
    target=row_dict.pop(VAR_TARGET)
    target = tf.one_hot(tf.cast(target,tf.int64),10)
    target = tf.cast(target,tf.float32)
    return(row_dict,target)

## Use Tensorflow I/O to Read Batches from BigQuery

Setup TensorFlow_IO client > session > table + table.map
- https://www.tensorflow.org/io/api_docs/python/tfio/bigquery/BigQueryClient

In [23]:
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{PROJECT_ID}",
        project_id = PROJECT_ID,
        table_id = f"{DATANAME}_prepped",
        dataset_id = DATANAME,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training

In [24]:
train = bq_reader('TRAIN').parallel_read_rows().map(transTable).shuffle(BATCH_SIZE*3).batch(BATCH_SIZE)
validate = bq_reader('VALIDATE').parallel_read_rows().map(transTable).batch(BATCH_SIZE)
test = bq_reader('TEST').parallel_read_rows().map(transTable).batch(BATCH_SIZE)

Review a single batch of the train data:

In [25]:
for a, b in train.take(1):
    columns=list(a.keys())
    print('columns:\n',columns)
    print('\ntarget:\n',b)

columns:
 ['p0', 'p1', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p2', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p3', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p4', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p5', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p6', 'p60', 'p61', 'p62', 'p63', 'p7', 'p8', 'p9']

target:
 tf.Tensor(
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [

---
## Train the Model In The Notebook

Define the Model:

In [26]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)
model = tf.keras.Model(
    inputs = [v for v in feature_layer_inputs.values()],
    outputs = tf.keras.layers.Dense(10, activation = tf.nn.softmax)(feature_layer_outputs)
)
model.compile(
    optimizer = 'sgd',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)
#tf.keras.utils.plot_model(model, show_shapes = True, show_dtype = True)

In [27]:
#model.summary()

Fit the Model:

In [28]:
history = model.fit(train, epochs = 25, validation_data = validate)

Epoch 1/25
49/49 [==============================] - 3s 53ms/step - loss: 2.8710 - accuracy: 0.5503 - val_loss: 1.7381 - val_accuracy: 0.6556
Epoch 2/25
49/49 [==============================] - 2s 39ms/step - loss: 0.6488 - accuracy: 0.8340 - val_loss: 0.7009 - val_accuracy: 0.8222
Epoch 3/25
49/49 [==============================] - 2s 45ms/step - loss: 0.4617 - accuracy: 0.8809 - val_loss: 0.6590 - val_accuracy: 0.8222
Epoch 4/25
49/49 [==============================] - 2s 42ms/step - loss: 0.3504 - accuracy: 0.9070 - val_loss: 0.4153 - val_accuracy: 0.8833
Epoch 5/25
49/49 [==============================] - 2s 34ms/step - loss: 0.3131 - accuracy: 0.9146 - val_loss: 0.4358 - val_accuracy: 0.8611
Epoch 6/25
49/49 [==============================] - 2s 35ms/step - loss: 0.2312 - accuracy: 0.9311 - val_loss: 0.3779 - val_accuracy: 0.9000
Epoch 7/25
49/49 [==============================] - 2s 38ms/step - loss: 0.2152 - accuracy: 0.9366 - val_loss: 0.3035 - val_accuracy: 0.9111
Epoch 8/25
49

In [29]:
history.history['loss'][-1]

0.07243506610393524

Evaluate the model with the test data:

In [30]:
loss, accuracy = model.evaluate(test)

6/6 [==============================] - 0s 12ms/step - loss: 0.1042 - accuracy: 0.9636


In [31]:
loss, accuracy = model.evaluate(validate)

6/6 [==============================] - 0s 13ms/step - loss: 0.2034 - accuracy: 0.9278


In [32]:
loss, accuracy = model.evaluate(train)

49/49 [==============================] - 1s 24ms/step - loss: 0.0633 - accuracy: 0.9828


Create Prediction from a batch of the test data and review first row:

In [33]:
model.predict(test.take(1))[0]

array([9.9956220e-01, 2.7984512e-07, 2.6025486e-06, 5.3722726e-10,
       2.1185444e-04, 1.6432656e-08, 2.2245137e-04, 2.4398972e-10,
       3.1432586e-07, 2.2100301e-07], dtype=float32)

---
## Serving

### Save The Model

In [34]:
model.save(URI)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


2021-09-19 12:30:22.503001: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: gs://statmike-mlops/digits/models/04a/assets
INFO:tensorflow:Assets written to: gs://statmike-mlops/digits/models/04a/assets


### Upload The Model

In [35]:
model = aiplatform.Model.upload(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = URI,
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/691911073727/locations/us-central1/models/8309625097614786560/operations/3000425120322813952
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/691911073727/locations/us-central1/models/8309625097614786560
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/691911073727/locations/us-central1/models/8309625097614786560')


In [36]:
model.display_name

'04a_digits_20210919122627'

### Create An Endpoint

In [37]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/691911073727/locations/us-central1/endpoints/1552879854326644736/operations/1568280438818996224
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/691911073727/locations/us-central1/endpoints/1552879854326644736
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/691911073727/locations/us-central1/endpoints/1552879854326644736')


In [38]:
endpoint.display_name

'04a_digits_20210919122627'

### Deploy Model To Endpoint

In [39]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/691911073727/locations/us-central1/models/8309625097614786560 to Endpoint : projects/691911073727/locations/us-central1/endpoints/1552879854326644736
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/691911073727/locations/us-central1/endpoints/1552879854326644736/operations/8814572239258124288
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/691911073727/locations/us-central1/endpoints/1552879854326644736


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [40]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME} LIMIT 10").to_dataframe()

In [41]:
pred.head(4)

,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p56,p57,p58,p59,p60,p61,p62,p63,target,target_OE
0,0.0,5.0,16.0,15.0,5.0,0.0,0.0,0.0,0.0,2.0,...,0.0,6.0,16.0,16.0,16.0,16.0,7.0,0.0,2,Even
1,0.0,5.0,16.0,12.0,1.0,0.0,0.0,0.0,0.0,5.0,...,0.0,8.0,16.0,16.0,16.0,16.0,4.0,0.0,2,Even
2,0.0,5.0,15.0,16.0,6.0,0.0,0.0,0.0,0.0,11.0,...,0.0,6.0,16.0,16.0,16.0,13.0,3.0,0.0,2,Even
3,0.0,4.0,15.0,15.0,8.0,0.0,0.0,0.0,0.0,8.0,...,0.0,7.0,14.0,11.0,0.0,0.0,0.0,0.0,2,Even


In [48]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET])]].to_dict(orient='records')[0]
#newob

In [49]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [50]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[[2.46273196e-10, 0.00158410147, 0.997878194, 1.5257192e-06, 2.3139702e-12, 1.76742205e-07, 7.47057638e-05, 2.49122e-10, 0.000460077048, 1.18600212e-06]], deployed_model_id='2499014008074403840', explanations=None)

In [51]:
prediction.predictions[0]

[2.46273196e-10,
 0.00158410147,
 0.997878194,
 1.5257192e-06,
 2.3139702e-12,
 1.76742205e-07,
 7.47057638e-05,
 2.49122e-10,
 0.000460077048,
 1.18600212e-06]

In [52]:
np.argmax(prediction.predictions[0])

2

### Get Predictions: REST

In [53]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [54]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      2.46273196e-10,
      0.00158410147,
      0.997878194,
      1.5257192e-06,
      2.3139702e-12,
      1.76742205e-07,
      7.47057638e-05,
      2.49122e-10,
      0.000460077048,
      1.18600212e-06
    ]
  ],
  "deployedModelId": "2499014008074403840"
}


### Get Predictions: gcloud (CLI)

In [55]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[2.46273196e-10, 0.00158410147, 0.997878194, 1.5257192e-06, 2.3139702e-12, 1.76742205e-07, 7.47057638e-05, 2.49122e-10, 0.000460077048, 1.18600212e-06]]


---
## Remove Resources
see notebook "XX - Cleanup"